# Computer Vision Recap - Segmentation

## Lecture 13: Isotropic Nonlinear Diffusion

### Isotropic Nonlinear Diffusion


Avoid drawbacks of linear diffusion: delocalisation and blurring of edges
$$ \partial_t u = div( g(|\nabla u|^2) \; \nabla u ) $$

Diffusivity $g(|\nabla u|^2) := \frac{1}{1+|\nabla u|^2/\lambda^2}$ decreasing in $|\nabla u|^2$ 


<img src="./images/isodiff.png" width=600>
<br>

- Forward diffusion if $\Phi'(u_x) > 0$ for $|\nabla u| < \lambda$
- Backward diffusion if $\Phi'(u_x) < 0$ for $|\nabla u| > \lambda$

### Regularised Nonlinear Diffusion

- Reduce staircasing artifacts and problems with noise

$$ \partial_t u = div( g(|\nabla u_\sigma|^2) \; \nabla u ) $$ with $$ u_\sigma = K_\sigma*u $$

<br>

Theoretical Results

- Stability under perturbations of initial image
- Preservation of average grey value
- Maximum-minimum principle
- Stable
- Image converges to average grey level

<br><br>
## Lecture 14: Anisotropic Nonlinear Diffusion

**Motivation**
- Isotropic nonlinear diff. have difficulties with noisy images and cannot enhance coherent flow-like structures
- We want to smooth only along edge directions but no across. 

<br>

### Edge-enhancing Anisotropic Diffusion (EED)

**Goal**
- Smooth prefently within regions
- Inhibit diffusion perpendicular to edges

$$ D(\nabla u_\sigma) = (v_1 \;\; v_2) \begin{bmatrix} \lambda_1&0\\0&\lambda_2 \end{bmatrix} (v_1 \;\; v_2)^\top = \lambda_1 v_1 v_1^\top + \lambda_2 v_2 v_2^\top $$

- $\lambda_1 = g(|\nabla u|^2)$ (diffusion across edge), $v_1 \parallel \nabla u_\sigma$
- $\lambda_2 = 1$ (diffusion along edge), $v_2 \perp \nabla u_\sigma$ 

<img src="./images/EED.png" width=300>

<br>

**Problems**:
- Large convolution kernels cancel adjacent gradients with the same direction but opposite orientation
- EED is not good for closing interrupted lines

**Analysing Local Structure**
- small $\sigma$: sensitive to noise
- big $\sigma$: cancellation of adjacent gradients with opposite directions

<br>

### Coherence-Enhancing Anisotropic Diffusion (CED)

Smooth only along the edges: $(\mu_1-\mu_2)^2 >> 0$, but not when we have corners or flat areas ($\mu_1\approx\mu_2$)

basically a 1-D smoothing along flow-like structures

- smooth mainly along coherent structures (i.e. along $v_2 \perp \nabla u_\sigma$)
- increase smoothing with coherence $(\mu_1-\mu_2)^2$ 

**Choice of Diffusion Tensor $D(J_\rho)$**

Coherence (anisotropy) $c=(\mu_1-\mu_2)^2$ are useful to analyse the gradient covariance (how allongated is the covariance?)

- eigenvectors from $J_\rho(\nabla u_\sigma) = K_\rho * (\nabla u_\sigma \nabla u_\sigma^\top)$
- Across coherent structures: $\lambda_1 := \alpha (small \; \alpha >0)$ 
- Along coherent structures: $\lambda_2 := \alpha + (1-\alpha) \exp( \frac{-C}{(\mu_1-\mu_2)^2} )$ 

<img src="./images/CEAD.png" width=300>


<br><br>
## Lecture 15: Mumford-Shah Model (Segmentation)



### Mumford-Shah Model

Seeks segmentation (u,K) of some initial image $f:\Omega\rightarrow \mathbb R$ as the minimizer of

$$ E_{MS}(u,K) = \int_\Omega (u-f)^2 dx dy + \alpha \int_{\Omega\backslash K} |\nabla u|^2 dx dy + \lambda l(K)$$

1. Penalises deviations from original image
2. Penalises variation within each segment
3. Penalises edge length

matematically extremely challenging

<br>

### Mumford-Shah Cartoon Model

Approximation for $\alpha\rightarrow \infty$, which implies that the segmented image is constant in each segment ("cartoon"):
$$ E_{MS}(u,K) = \int_{\Omega\backslash K} (u-f)^2 dx dy + \lambda l(K)$$

which is sufficient to control the homogeneity within each segment and also minimize the length of the boundaries

#### Approximation method for minimization

A segmentation $K$ is called **2-normal** if every segmentation $K'$, that results from merging two neighbouring segments of $K$ has a larger energy:
$$E(K') > E(K)$$
This means a 2-normal segmentation is a local minimum

**Idea:** approximate global minimiser of cartoon model by local minimiser resulting from the restriction to 2-normal segmentations. Create 2-normal segmentations by region merging

Merging two regions yields:
$$ E(K\backslash \partial(\Omega_i,\Omega_j)) = E(K) + \frac{|\Omega_i|\cdot |\Omega_j|}{|\Omega_i|+|\Omega_j|} (u_i-u_j)^2  - \lambda l(\partial (\Omega_i,\Omega_j))$$

Solving $E(K\backslash \partial(\Omega_i,\Omega_j)) - E(K) = 0$ yields:
$$ \lambda_{i,j} = \frac{ \frac{|\Omega_i|\cdot |\Omega_j|}{|\Omega_i|+|\Omega_j|} (u_i-u_j)^2 }{l(\partial (\Omega_i,\Omega_j))} $$

for $\lambda > \lambda_{i,j}$: decrease of energy when mergin $i$ and $j$ <br>
for $\lambda < \lambda_{i,j}$: increase of energy when mergin $i$ and $j$

<br>

### The Ambrosio-Tortorelli Mode



Approximation of the Mumford-Shah functional by:
$$ E_{MS-AT}(u,v) := \int_{\Omega} \left(  (u-f)^2 + \alpha\cdot v^2\cdot |\nabla u|^2 + \beta\cdot \left(c|\nabla v|^2+\dfrac{(1-v)^2}{4c}\right)  \right) dx dy$$

where v(x,y) is an edge indicator function ($v\rightarrow 0$ at edges and 1 within region)

- **Advantage**: smooth unkown function $v$ is mathm. more convenient than an unkown edge set $K$ 

<br>

Solving with Euler-Lagrange equations yields:
$$ 0 = \frac{1}{\alpha} (u-f) - div(v^2 \nabla u) $$
$$ 0 = \frac{\alpha}{\beta} v |\nabla u|^2 - \frac{1-v}{4c} - c\Delta v $$ 

with boundary conditions:
$$ 0 = n \cdot \nabla u \quad \quad 0=n\cdot\nabla v  $$

<br>

- The MS-AT energy is not jointly convex in $(u,v)$ and therefore may have multiple minima
- Consequently, the diffusion-reaction system may get stuck in a local minima

<br>

The Ambrosio-Tortorelli are related to the Perona-Malik (nonlinear isotropic diffusion) filter

## Lecture 16: Continuous Scaled Morphology, Schock Filters

### Classical Morphology

Typical structuring elements B: discs,s squares of ellipses

- Dilation: $(f \oplus B) (x) := \sup \{ f(x-y) | y \in B \}$
- Erosion: $(f\ominus B)(x) := \inf \{  f(x+y) | y \in B \}$
- Opening:  $(f \circ B)(\boldsymbol{x}):=((f \ominus B) \oplus B)(\boldsymbol{x})$ 
- Closing: $(f \bullet B)(\boldsymbol{x}):=((f \oplus B) \ominus B)(\boldsymbol{x})$

<br>

### Morphological Curve Evolution

Consider a binary image object with boundary curve $c_0 = (x(p),y(p))$

- Dilate with a disc-shaped structuring elements of continuously increasing radius t. Then the resulting curve c(t) follows the evolution (propagation in outer normal direction)
$$ \partial_t c = n$$

- Erosion it with a disc-shaped propagates in inner normal direction normal 
$$ \partial_t c = -n  $$

where $n=\frac{\nabla u}{|\nabla u|}$ and with initial condition $c(o)=c_0$

<br>

### From curve evolution to image evolution

Consider some curve evolution with speed function $\beta$ depending on the curvature $\kappa$
$$ \partial_t c = \beta(\kappa) \cdot n $$

<br> 

Level set: embed $c$ as a level line into some smooth scalar-valued image $u$, using the **signed distance function**:

$$u(\boldsymbol{x}, t):=\left\{\begin{aligned} \operatorname{dist}(\boldsymbol{x}, \boldsymbol{c}(t)) &\quad (\boldsymbol{x} \text { inside } \boldsymbol{c}) \\-\operatorname{dist} (\boldsymbol{x}, \boldsymbol{c}(t)) & \quad (\mathrm{else}) \end{aligned}\right.$$

Then the zero crossing of $u$ are given by: $$ u(c,t)=0 $$

<img src="./images/curveevo.png" width=600>

<br>

**Main result:** Curve evolution $\partial_t c = \pm n$ leads to image evolutions $\partial_t u = \pm |\nabla u|$. (+ for dilation, - for erosion)

<img src="./images/dilationerosion.png" width=400>



<br>

### Shock Filtering

- Mercury Dilation around maxima (regions where $\Delta u < 0$)
- Erosion around minima (regions where $\Delta u > 0$)

Shock equation:
$$ \partial_t u = -sgn(\Delta u) \; |\nabla u| $$

Create shock at edge locations where $\Delta u=0$


<img src="./images/shockfilter.png" width=400>

<br><br>

### Coherence enhancing shock filter:
$$ \partial_t u = -sgn(u_{ww}) \; |\nabla u| $$
where $w$ is the normalised dominant eigenvector of the structure tensor $J_\rho(\nabla u)$ 


<br><br>

## Lecture 17: Mean Curvature Motion

### Motivation:

Dilation/Erosion
$$ \partial_t u = \pm |\nabla u| $$
propagates level curves (isophotes) with constant speed in outer/inner normal direction
$$\partial_t c = \pm n$$

However, the framework also allows 
$$ \partial_t c = \beta(\kappa) n \quad \Rightarrow \quad \partial_t u = \beta(\kappa) |\nabla u|$$

where $\beta(\kappa)$ is a curvature dependent speed function

<br>

### Mean Curvature Motion as Image Evolution

Consider
$$\partial_t u = \Delta u = \partial_{xx} u + \partial_{yy} u = \partial_{\eta\eta} u + \partial_{\xi\xi} u$$

with $\partial_{\eta\eta} u$ along the flowlines and  $\partial_{\xi\xi} u$ along the isophotes ($n \parallel \nabla u, \quad \xi \perp \nabla u$)

A PDE that performs only anisotropic smoothing along isophotes is called Mean Curvature Mtotion (MCM):
$$ \begin{align} \partial_t u &= \partial_{\xi\xi} u \\ &= |\nabla u| \, div \left( \frac{\nabla u}{|\nabla u|} \right) \\ &= curv(u) \, |\nabla u|  \end{align}$$

where curv(u) denotes the curvature of an isophote (inverse radius of osculating circle)

- $sgn(curv(u))$ determines if it will locally act like dilation of erosion
- the larger $|\nabla u|$ the larger the diffusion


Time for shrinking a circle of radius $\sigma$ to a point: $T=\frac{1}{2} \sigma^2$


<img src="./images/MCM.png" width=300>






<br><br>

## Lecture 18: Self-Snakes, Active Contours

### Self-Snakes

#### Motivation 

MCM dos not allow to enhance edges

#### Formulation

Modification of MCM:  $\;\partial_t u = |\nabla u| \; div\left( \frac{\nabla u}{|\nabla u|} \right)$, to:
$$ \partial u = |\nabla u| \; div\left( g(|\nabla u_\sigma|^2) \frac{\nabla u}{|\nabla u|} \right) $$

We can rewrite this equation as:
$$\begin{aligned} \partial_{t} u &=|\nabla u| \operatorname{div}\left(g\left(\left|\nabla u_{\sigma}\right|^{2}\right) \frac{\nabla u}{|\nabla u|}\right) \\ &=|\nabla u|\left(g\left(\left|\nabla u_{\sigma}\right|^{2}\right) \operatorname{div}\left(\frac{\nabla u}{|\nabla u|}\right)+\nabla g\left(\left|\nabla u_{\sigma}\right|^{2}\right)^{\top} \frac{\nabla u}{|\nabla u|}\right) \\ &=g\left(\left|\nabla u_{\sigma}\right|^{2}\right)|\nabla u| \operatorname{div}\left(\frac{\nabla u}{|\nabla u|}\right)+ \color{red}{ \nabla g\left(\left|\nabla u_{\sigma}\right|^{2}\right)^{\top} \nabla u } \end{aligned}$$

The $\color{red}{red}$ term is responsible for edge enhancement because it resembles a shock filter: $\partial_t u = -sgn(\Delta u) |\nabla u|$



### Geodesic Active Contours

**Goal**: shrink a manually specified contour towards an object boundary of some image

Image envolution to make an isophote on image $f$ shrinks towards an object boundary on image $h$:

$$ \begin{aligned} &\partial_{t} u=|\nabla u| \operatorname{div}\left(g\left( \color{red}{ \left|\nabla h_{\sigma}\right|^{2} } \right) \frac{\nabla u}{|\nabla u|}\right)\\ &u(t=0)=f \end{aligned} $$

**where the edge-stopping function g inhibits evolution at edges of h (not f)**

<br>

We can also rewrite the image evolution as:

$$\begin{aligned} \partial_{t} u &=|\nabla u| g \operatorname{div}\left(\frac{\nabla u}{|\nabla u|}\right)+|\nabla u| \nabla g^{\top} \frac{\nabla u}{|\nabla u|} \\  &=g \operatorname{div}\left(\frac{\nabla u}{|\nabla u|}\right)|\nabla u|+\nabla g^{\top} \frac{\nabla u}{|\nabla u|}|\nabla u| \end{aligned}$$

it evolves its isophotes under
$$ \partial_t c = g \, \kappa \, n - (\nabla g^\top \, n)\, n $$



<br><br>

### Region-based Active Contours

Use region-based information instead of edge-based  information allows us to be more robust under noise, present a better convergence behaviour.

### The Chan-Vese Model


$$\begin{aligned} E_{\mathrm{CV}}\left(\boldsymbol{c}, u_{\mathrm{in}}, u_{\mathrm{out}}\right) &=\int\left(f(\boldsymbol{x})-u_{\mathrm{in}}\right)^{2} d \boldsymbol{x}+\int\left(f(\boldsymbol{x})-u_{\mathrm{out}}\right)^{2} d \boldsymbol{x}+\lambda l(\boldsymbol{c}) \\ & \text { inside } \boldsymbol{c} \quad\quad\quad\quad\quad \text { outside } \boldsymbol{c} \end{aligned}$$

where uin and uout are the arithmetic means of f(x) inside resp. outside c, and l(c) is the contour length.

which is similar to the Mumford-Shah Cartoon model, equivalent for edge sets K that separate $\Omega$ into only two regions.

<br>

Level set formulation:

$$ E_{I} \left(v, u_{\mathrm{in}}, u_{\mathrm{out}}\right) = \int_{\Omega}\left(f-u_{\mathrm{in}}\right)^{2} H(v) d \boldsymbol{x} + \int_{\Omega}\left(f-u_{\mathrm{out}}\right)^{2}(1-H(v)) d \boldsymbol{x} +\lambda \int_{\Omega}|\nabla H(v)| d x $$

with the Heaviside function $H(z) = 1 \, (z\ge0), \quad 0 \, (z<0)$

$u_{in}$/$u_{out}$ are the means of $f(x)$ inside/outside and $v$ is the level set image, which defines the zero level line of the segmented image

<br>

The Euler-Lagrange equation can be solved using gradient descent:
$$\partial_{t} v=H_{\varepsilon}^{\prime}(v)\left(\left(f-u_{\mathrm{out}}\right)^{2}-\left(f-u_{\mathrm{in}}\right)^{2}+\lambda \operatorname{div}\left(\frac{\nabla v}{|\nabla v|}\right)\right)$$

How to solve: solve gradient descent step for $v$ and then update means $u_{in},u_{out}$

<img src="./images/chanvese.png" width=600>


